In [2]:
# ------ Import necessary packages ----
# 必要なパッケージのインポート
from collections import defaultdict
	

from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import networkx as nx
	
import matplotlib
matplotlib.use("agg")
from matplotlib import pyplot as plt

In [9]:
	# ------- Set up our graph -------
# 	グラフのセットアップ

	# Create empty graph
#     空のグラフ用意
	G = nx.Graph()
	

	# Add edges to the graph (also adds nodes)
#     辺をいれる同様に頂点を形成
	G.add_edges_from([(1,2),(1,3),(2,4),(3,4),(3,5),(4,5)])
print( G.edges)

[(1, 2), (1, 3), (2, 4), (3, 4), (3, 5), (4, 5)]


In [4]:
	# ------- Set up our QUBO dictionary -------
# 	QUBO 辞書をセットアップ

	# Initialize our Q matrix
#     Q行列を準備
	Q = defaultdict(int)
	

	# Update Q matrix for every edge in the graph
#     それぞれのQ行列をアップデート
	for i, j in G.edges:
	    Q[(i,i)]+= -1
	    Q[(j,j)]+= -1
	    Q[(i,j)]+= 2
    
    print(Q)
    


defaultdict(<class 'int'>, {(1, 1): -1, (2, 2): -1, (1, 2): 2})
defaultdict(<class 'int'>, {(1, 1): -2, (2, 2): -1, (1, 2): 2, (3, 3): -1, (1, 3): 2})
defaultdict(<class 'int'>, {(1, 1): -2, (2, 2): -2, (1, 2): 2, (3, 3): -1, (1, 3): 2, (4, 4): -1, (2, 4): 2})
defaultdict(<class 'int'>, {(1, 1): -2, (2, 2): -2, (1, 2): 2, (3, 3): -2, (1, 3): 2, (4, 4): -2, (2, 4): 2, (3, 4): 2})
defaultdict(<class 'int'>, {(1, 1): -2, (2, 2): -2, (1, 2): 2, (3, 3): -3, (1, 3): 2, (4, 4): -2, (2, 4): 2, (3, 4): 2, (5, 5): -1, (3, 5): 2})
defaultdict(<class 'int'>, {(1, 1): -2, (2, 2): -2, (1, 2): 2, (3, 3): -3, (1, 3): 2, (4, 4): -3, (2, 4): 2, (3, 4): 2, (5, 5): -2, (3, 5): 2, (4, 5): 2})


In [8]:
	# ------- Run our QUBO on the QPU -------
#     QUBOとQPUを走らせる
	# Set up QPU parameters
#     QPUのパラメータをセットアップ
	chainstrength = 8
	numruns = 10
	

	# Run the QUBO on the solver from your config file
#  　CONFIGファイルのソルバーからQUBO を走らせる
	sampler = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))
	response = sampler.sample_qubo(Q, chain_strength=chainstrength, num_reads=numruns)
	energies = iter(response.data())
for i in energies:
    print(i)

Sample(sample={1: 0, 2: 1, 3: 1, 4: 0, 5: 0}, energy=-5.0, num_occurrences=1, chain_break_fraction=0.0)
Sample(sample={1: 1, 2: 0, 3: 0, 4: 1, 5: 0}, energy=-5.0, num_occurrences=3, chain_break_fraction=0.0)
Sample(sample={1: 0, 2: 1, 3: 1, 4: 0, 5: 1}, energy=-5.0, num_occurrences=2, chain_break_fraction=0.0)
Sample(sample={1: 1, 2: 0, 3: 0, 4: 1, 5: 1}, energy=-5.0, num_occurrences=1, chain_break_fraction=0.0)
Sample(sample={1: 1, 2: 1, 3: 0, 4: 0, 5: 1}, energy=-4.0, num_occurrences=1, chain_break_fraction=0.0)
Sample(sample={1: 0, 2: 1, 3: 0, 4: 0, 5: 1}, energy=-4.0, num_occurrences=1, chain_break_fraction=0.0)
Sample(sample={1: 1, 2: 0, 3: 1, 4: 1, 5: 0}, energy=-4.0, num_occurrences=1, chain_break_fraction=0.0)


In [5]:
	# ------- Print results to user -------
#     結果を出力
	print('-' * 60)
	print('{:>15s}{:>15s}{:^15s}{:^15s}'.format('Set 0','Set 1','Energy','Cut Size'))
	print('-' * 60)
	for line in response:
	    S0 = [k for k,v in line.items() if v == 0]
	    S1 = [k for k,v in line.items() if v == 1]
	    E = next(energies).energy
	    print('{:>15s}{:>15s}{:^15s}{:^15s}'.format(str(S0),str(S1),str(E),str(int(-1*E))))


------------------------------------------------------------
          Set 0          Set 1    Energy        Cut Size    
------------------------------------------------------------
      [2, 3, 5]         [1, 4]     -5.0             5       
         [1, 4]      [2, 3, 5]     -5.0             5       
         [2, 3]      [1, 4, 5]     -5.0             5       
         [3, 4]      [1, 2, 5]     -4.0             4       
      [2, 3, 4]         [1, 5]     -4.0             4       


In [13]:
	# ------- Display results to user -------
	# Grab best result
	# Note: "best" result is the result with the lowest energy
	# Note2: the look up table (lut) is a dictionary, where the key is the node index
	#   and the value is the set label. For example, lut[5] = 1, indicates that
	#   node 5 is in set 1 (S1).
	lut = response.lowest().first.sample
	

	# Interpret best result in terms of nodes and edges
#     この処理のあと、D-Waveの管理画面にログインするとUsage Summaryがでてくるが、少し使ったことがわかる
	S0 = [node for node in G.nodes if not lut[node]]
	S1 = [node for node in G.nodes if lut[node]]
	cut_edges = [(u, v) for u, v in G.edges if lut[u]!=lut[v]]
	uncut_edges = [(u, v) for u, v in G.edges if lut[u]==lut[v]]

print(cut_edges)
print(uncut_edges)

[(1, 2), (1, 3), (2, 4), (3, 4), (3, 5)]
[(4, 5)]


In [8]:
	# Display best result
	pos = nx.spring_layout(G)
	nx.draw_networkx_nodes(G, pos, nodelist=S0, node_color='r')
	nx.draw_networkx_nodes(G, pos, nodelist=S1, node_color='c')
	nx.draw_networkx_edges(G, pos, edgelist=cut_edges, style='dashdot', alpha=0.5, width=3)
	nx.draw_networkx_edges(G, pos, edgelist=uncut_edges, style='solid', width=3)
	nx.draw_networkx_labels(G, pos)
	

	filename = "maxcut_plot.png"
	plt.savefig(filename, bbox_inches='tight')
	print("\nYour plot is saved to {}".format(filename))



Your plot is saved to maxcut_plot.png
